# Notebook Contents

- [Imports](#Imports)
- [Data](#Data)
- [Data Cleaning](#Data-Cleaning)
- [Preprocessing](#Preprocessing)
    - [Multicolinearity - VIF](#Multicolinearity---VIF)
- [Features](#Features)
- [Random Forest Modeling](#Random-Forest-Modeling)
    - [4-Seam](#Linear-Regression---4-Seam)
    - [Cutter](#Linear-Regression---Cutter)
    - [Sinker](#Linear-Regression---Sinker)
    - [Slider](#Linear-Regression---Slider)
    - [Curveball](#Linear-Regression---Curveball)
    - [Changeup](#Linear-Regression---Changeup)

# Imports

In [93]:
from pprint import pprint
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, k_means
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')

# Data

In [94]:
data = pd.read_csv('../data/model-pitches.csv', index_col = [0])
data.dropna(subset = ['pitch_type', 'velo', 'spin_rate', 'pfx_-x', 
                      'release_extension', 'delta_run_exp'], inplace = True)

data['inning_topbot'] = data.inning_topbot.map({'Top': 0, 'Bot': 1})
data['on_1b'] = [1 if x > 1 else 0 for x in data['on_1b']]
data['on_2b'] = [1 if x > 1 else 0 for x in data['on_2b']]
data['on_3b'] = [1 if x > 1 else 0 for x in data['on_3b']]

data['home_runs'] = data['post_home_score'] - data['home_score']
data['away_runs'] = data['post_away_score'] - data['away_score']
data['runs'] = data['home_runs'] + data['away_runs']

re = pd.read_csv('../data/run_expectancy_table.csv', index_col = [0])
data = pd.merge(data, re, how = 'left', on = ['on_1b', 'on_2b', 'on_3b', 'outs_when_up'])

data['re_end_state'] = data['delta_run_exp'] + data['re']
data['re24'] = data['re_end_state'] - data['re'] + data['runs']

count_woba = pd.read_csv('../data/count_woba_value.csv', index_col = [0])
data = pd.merge(data, count_woba, how = 'left', on = ['pitch_count'])
data['count_woba_diff'] = -data['count_woba_value'].diff(1)
data['count_woba_after'] = data['count_woba_value'] + data['count_woba_diff']
data['count_woba_diff'].fillna(0, inplace = True)
data['count_woba_after'].fillna(0, inplace = True)
data['rv_count'] = round((data['count_woba_diff']) / 1.15, 3)

pd.set_option('max_columns', None)
print(data.shape)
data.head()

(705403, 57)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,pitch_count,stand,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,xba,xwobacon,woba_value,woba_denom,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,delta_run_exp,home_runs,away_runs,runs,re,re_end_state,re24,count_woba_value,count_woba_diff,count_woba_after,rv_count
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,1.40,6.80,6.5,54.03,0.69,-0.69,2.83,X,1,2,1-2,R,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.174,0.158,0.0,1.0,0.0,0.0,61,4,9,0,5,0,5,0,0,0,0,2,-0.073,0,0,0,0.115,0.042,-0.073,0.233,0.000,0.000,0.000
1,"Smith, Will",L,SL,80.6,2254.0,315.0,9.24,5.76,27.965261,81.2,1.60,6.64,6.4,54.15,0.71,-0.71,2.62,S,1,1,1-1,R,foul,NaN,108.0,75.3,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,9,0,5,0,5,0,0,0,0,2,-0.027,0,0,0,0.115,0.088,-0.027,0.313,-0.080,0.233,-0.070
2,"Smith, Will",L,CU,75.5,1940.0,328.0,7.80,-6.12,25.695364,75.2,1.46,6.88,6.2,54.34,0.04,-0.04,2.46,S,1,0,1-0,R,foul,NaN,157.0,83.5,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,9,0,5,0,5,0,0,0,0,2,-0.020,0,0,0,0.115,0.095,-0.020,0.374,-0.061,0.313,-0.053
3,"Smith, Will",L,CU,75.0,2017.0,330.0,8.28,-8.28,26.893333,74.5,1.53,6.83,5.9,54.61,-2.10,2.10,3.89,B,0,0,0-0,R,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,1,9,0,5,0,5,0,0,0,0,2,0.016,0,0,0,0.115,0.131,0.016,0.331,0.043,0.374,0.037
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,1.49,6.66,6.3,54.15,0.31,-0.31,2.80,X,1,0,1-0,L,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.100,0.090,0.0,1.0,0.0,0.0,60,2,9,0,5,0,5,0,0,0,0,1,-0.189,0,0,0,0.298,0.109,-0.189,0.374,-0.043,0.331,-0.037


In [95]:
data.events.value_counts()

field_out                     81369
strikeout                     41807
single                        24848
walk                          14978
double                         7815
home_run                       5892
field_error                    1230
sac_bunt                        761
triple                          667
double_play                     381
caught_stealing_2b              146
strikeout_double_play           108
catcher_interf                   61
other_out                        41
sac_fly_double_play              17
caught_stealing_3b                8
pickoff_2b                        6
pickoff_1b                        6
wild_pitch                        5
triple_play                       5
caught_stealing_home              4
pickoff_caught_stealing_2b        4
pickoff_3b                        2
sac_bunt_double_play              2
passed_ball                       1
stolen_base_2b                    1
pickoff_caught_stealing_3b        1
Name: events, dtype: int64

In [96]:
#lin_weight = data['re24'].mean()
#data['lin_weight'] = lin_weight
#data.head()

Using this wOBA by count, we can calculate the value of points scored by count.

(count wOBA after pitching – count wOBA before pitching) / wOBAscale (≈1.15 in Statcast csv data)

First, when the count changes, the actual RAA is calculated as:

(wOBA of the count after the pitch – wOBA of the count before the pitch) / 1.15

If a batted ball occurs, then this is used to calculate RAA:

(xwOBAvalue – wOBA of the count before the pitch) / wOBAscale

# Preprocessing

### Multicolinearity - VIF
**Independent Variables:** Velocity, Spin Rate, VB, HB, Release Extension, Horizontal Release Position, Vertical Release Position, Horizontal Plate Coords, Vertical Plate Coords

**Dependent Variable:** rv_count

In [97]:
features = data[['velo', 'spin_rate', 'pfx_-x', 'pfx_z', 'release_extension', 
                 'release_pos_x', 'release_pos_z', 'plate_x', 'plate_z', 'rv_count',
                 'pitch_type', 'p_throws', 'stand']]

features_vif = features.select_dtypes([np.number])
vif_data = pd.DataFrame()
vif_data["feature"] = features_vif.columns

vif_data["VIF"] = [variance_inflation_factor(features_vif.values, i)
                   for i in range(len(features_vif.columns))]

vif_data.sort_values(by = 'VIF').head(10)

,feature,VIF
9,rv_count,1.011577
7,plate_x,1.110665
5,release_pos_x,1.454784
2,pfx_-x,1.466136
3,pfx_z,3.125169
8,plate_z,7.462887
1,spin_rate,50.832560
6,release_pos_z,103.379774
4,release_extension,152.725687
0,velo,277.711423


# Features

In [98]:
zero_outs = data.loc[data['outs_when_up'] == 0]
print('0 outs:', zero_outs.shape)
one_out = data.loc[data['outs_when_up'] == 1]
print('1 out:', one_out.shape)
two_outs = data.loc[data['outs_when_up'] == 2]
print('2 outs:', two_outs.shape)

0 outs: (244451, 57)
1 out: (232568, 57)
2 outs: (228384, 57)


In [99]:
ff = features.loc[features['pitch_type'] == 'FF']
fc = features.loc[features['pitch_type'] == 'FC']
fastball = ff.append(fc)
si = features.loc[features['pitch_type'] == 'SI']
fastball = fastball.append(si)
print('Fastball shape:', fastball.shape)
sl = features.loc[features['pitch_type'] == 'SL']
cu = features.loc[features['pitch_type'] == 'CU']
breaking_ball = sl.append(cu)
kc = features.loc[features['pitch_type'] == 'KC']
breaking_ball = breaking_ball.append(kc)
print('Breaking Ball:', breaking_ball.shape)
ch = features.loc[features['pitch_type'] == 'CH']
fs = features.loc[features['pitch_type'] == 'FS']
offspeed = ch.append(fs)
print('Off speed shape:', offspeed.shape)
rhp = features.loc[features['p_throws'] == 'R']
print('RHP shape:', rhp.shape)
lhp = features.loc[features['p_throws'] == 'L']
print('LHP shape:', lhp.shape)
rhp_rhh = features.loc[(features['p_throws'] == 'R') & (features['stand'] == 'R')]
print('RHP & RHH shape:', rhp_rhh.shape)
rhp_lhh = features.loc[(features['p_throws'] == 'R') & (features['stand'] == 'L')]
print('RHP & LHH shape:', rhp_lhh.shape)
lhp_rhh = features.loc[(features['p_throws'] == 'L') & (features['stand'] == 'R')]
print('LHP & RHH shape:', lhp_rhh.shape)
lhp_lhh = features.loc[(features['p_throws'] == 'L') & (features['stand'] == 'L')]
print('LHP & LHH shape:', lhp_lhh.shape)
rhp_fastball = fastball.loc[fastball['p_throws'] == 'R']
print('RHP Fastball shape:', rhp_fastball.shape)
lhp_fastball = fastball.loc[fastball['p_throws'] == 'L']
print('LHP Fastball shape:', lhp_fastball.shape)
rhp_breaking_ball = breaking_ball.loc[breaking_ball['p_throws'] == 'R']
print('RHP Breaking Ball shape:', rhp_breaking_ball.shape)
lhp_breaking_ball = breaking_ball.loc[breaking_ball['p_throws'] == 'L']
print('LHP Breaking Ball shape:', lhp_breaking_ball.shape)
rhp_offspeed = offspeed.loc[offspeed['p_throws'] == 'R']
print('RHP Offspeed shape:', rhp_offspeed.shape)
lhp_offspeed = offspeed.loc[offspeed['p_throws'] == 'L']
print('LHP Offspeed shape:', lhp_offspeed.shape)

Fastball shape: (406259, 13)
Breaking Ball: (207982, 13)
Off speed shape: (91162, 13)
RHP shape: (496498, 13)
LHP shape: (208905, 13)
RHP & RHH shape: (267548, 13)
RHP & LHH shape: (228950, 13)
LHP & RHH shape: (149824, 13)
LHP & LHH shape: (59081, 13)
RHP Fastball shape: (283224, 13)
LHP Fastball shape: (123035, 13)
RHP Breaking Ball shape: (152383, 13)
LHP Breaking Ball shape: (55599, 13)
RHP Offspeed shape: (60891, 13)
LHP Offspeed shape: (30271, 13)


# Random Forest Model

## Fastball RHP

#### 4-Seam, Cutter, Sinker

In [17]:
# params = {
#    'n_estimators': [25, 75],
#    'max_depth': [None, 5, 6, 7, 8],
#    'min_samples_split': [1, 2, 4],
#    'min_samples_leaf': [2, 5, 10],
#    'n_jobs': [-1]
#}
    
# gs = GridSearchCV(RandomForestRegressor(random_state = 10), param_grid = params)
# gs.fit(X_train, y_train)
# fastball_pred_r = gs.predict(X_test)

# print('Cross val score:', round((gs.best_score_), 4))

# gs.best_params_

In [19]:
features_fastball_r = rhp_fastball.select_dtypes([np.number])
X = features_fastball_r.drop(columns = ['rv_count'])
y = features_fastball_r['rv_count']

X_train, X_test, y_train, y_test = train_test_split(X, y)

rf = RandomForestRegressor()
rf.fit(X_train, y_train)
fastball_pred_r = rf.predict(X_test)

print('MSE:', round(metrics.mean_squared_error(y_test, fastball_pred_r), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, fastball_pred_r)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, fastball_pred_r), 4))

In [ ]:
print('Cross val score:', round(cross_val_score(rf, X_train, y_train).mean(), 4))

## Fastball LHP

#### 4-Seam, Cutter, Sinker

In [ ]:
features_fastball_l = lhp_fastball.select_dtypes([np.number])
X = features_fastball_l.drop(columns = ['rv_count'])
y = features_fastball_l['rv_count']

X_train, X_test, y_train, y_test = train_test_split(X, y)

params = {
    'n_estimators': [25, 75],
    'max_depth': [None, 5, 6, 7, 8],
    'min_samples_split': [1, 2, 4],
    'min_samples_leaf': [2, 5, 10],
    'n_jobs': [-1]
}
    
gs = GridSearchCV(RandomForestRegressor(random_state = 10), param_grid = params)
gs.fit(X_train, y_train)
fastball_pred_l = gs.predict(X_test)

print('Cross val score:', round((gs.best_score_), 4))
print('MSE:', round(metrics.mean_squared_error(y_test, fastball_pred_l), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, fastball_pred_l)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, fastball_pred_l), 4))
gs.best_params_

## Breaking Ball RHP

#### Slider, Curveball, Knuckle Curve

In [ ]:
features_bb_r = rhp_breaking_ball.select_dtypes([np.number])
X = features_bb_r.drop(columns = ['rv_count'])
y = features_bb_r['rv_count']

X_train, X_test, y_train, y_test = train_test_split(X, y)

params = {
    'n_estimators': [25, 75],
    'max_depth': [None, 5, 6, 7, 8],
    'min_samples_split': [1, 2, 4],
    'min_samples_leaf': [2, 5, 10],
    'n_jobs': [-1]
}
    
gs = GridSearchCV(RandomForestRegressor(random_state = 10), param_grid = params)
gs.fit(X_train, y_train)
bb_pred_r = gs.predict(X_test)

print('Cross val score:', round((gs.best_score_), 4))
print('MSE:', round(metrics.mean_squared_error(y_test, bb_pred_r), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, bb_pred_r)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, bb_pred_r), 4))
gs.best_params_

## Breaking Ball LHP

#### Slider, Curveball, Knuckle Curve

In [ ]:
features_bb_l = lhp_breaking_ball.select_dtypes([np.number])
X = features_bb_l.drop(columns = ['rv_count'])
y = features_bb_l['rv_count']

X_train, X_test, y_train, y_test = train_test_split(X, y)

params = {
    'n_estimators': [25, 75],
    'max_depth': [None, 5, 6, 7, 8],
    'min_samples_split': [1, 2, 4],
    'min_samples_leaf': [2, 5, 10],
    'n_jobs': [-1]
}
    
gs = GridSearchCV(RandomForestRegressor(random_state = 10), param_grid = params)
gs.fit(X_train, y_train)
bb_pred_l = gs.predict(X_test)

print('Cross val score:', round((gs.best_score_), 4))
print('MSE:', round(metrics.mean_squared_error(y_test, bb_pred_l), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, bb_pred_l)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, bb_pred_l), 4))
gs.best_params_

## Offspeed RHP

#### Changeup, Splitter

In [ ]:
features_os_r = rhp_offspeed.select_dtypes([np.number])
X = features_os_r.drop(columns = ['rv_count'])
y = features_os_r['rv_count']

X_train, X_test, y_train, y_test = train_test_split(X, y)

params = {
    'n_estimators': [25, 75],
    'max_depth': [None, 5, 6, 7, 8],
    'min_samples_split': [1, 2, 4],
    'min_samples_leaf': [2, 5, 10],
    'n_jobs': [-1]
}
    
gs = GridSearchCV(RandomForestRegressor(random_state = 10), param_grid = params)
gs.fit(X_train, y_train)
os_pred_r = gs.predict(X_test)

print('Cross val score:', round((gs.best_score_), 4))
print('MSE:', round(metrics.mean_squared_error(y_test, os_pred_r), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, os_pred_r)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, os_pred_r), 4))
gs.best_params_

## Offspeed LHP

#### Changeup, Splitter

In [ ]:
features_os_l = lhp_offspeed.select_dtypes([np.number])
X = features_os_l.drop(columns = ['rv_count'])
y = features_os_l['rv_count']

X_train, X_test, y_train, y_test = train_test_split(X, y)

params = {
    'n_estimators': [25, 75],
    'max_depth': [None, 5, 6, 7, 8],
    'min_samples_split': [1, 2, 4],
    'min_samples_leaf': [2, 5, 10],
    'n_jobs': [-1]
}
    
gs = GridSearchCV(RandomForestRegressor(random_state = 10), param_grid = params)
gs.fit(X_train, y_train)
os_pred_l = gs.predict(X_test)

print('Cross val score:', round((gs.best_score_), 4))
print('MSE:', round(metrics.mean_squared_error(y_test, os_pred_l), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, os_pred_l)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, os_pred_l), 4))
gs.best_params_